In [2]:
#패키지 임포트 
import os 
import warnings 

warnings.simplefilter(action ='ignore',category= FutureWarning)

import pandas as pd 
import numpy as np 
from scipy import stats


#시각화 패키지 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
if os.name == 'nt' :
    font_family = 'Malgun Gothic'
else:
    font_family = 'AppleGothic'

#~값이 깨지는 문제 해결을 위해 파라미터값 설정 
sns.set(font=font_family, rc = {"axes.unicode_minus" : False})

In [3]:
# 적합도 검정
# 4교대 근무, 1일 불량수 조사 , 각조의 불량 수 차이 여부 검정

observed_, expected_ = [15, 35, 24, 26], [25, 25, 25, 25]
ct_1 = pd.DataFrame([observed_, expected_], columns= ['A', 
                                                     'B',
                                                     'C',
                                                     'D'], 
                   index = ['관측빈도', '기대빈도'])
ct_1

,A,B,C,D
관측빈도,15,35,24,26
기대빈도,25,25,25,25


In [4]:
# 카이제곱 검정 통계랑 계산
observed = ct_1.loc['관측빈도']
expected= ct_1.loc['기대빈도']

chi_squared_test = (observed-expected).pow(2)/expected

In [5]:
chi_squared_statistic = chi_squared_test.sum()
print('카이제곱 통계랑', chi_squared_statistic)

카이제곱 통계랑 8.079999999999998


In [6]:
# 카이제곱 검정 수행- 적합도 검정
dof = (4-1) * (2-1)
chi_ = stats.chi2(dof)
chi_05 = chi_.ppf(0.95)
print('기각역 :', chi_05)


기각역 : 7.814727903251179


In [7]:
#카이제곱 통계량 기반검정
if chi_05 >= chi_squared_statistic:
    print('귀무가설 채택, 4개조의 불량수는 차이가 없다.')
else:
    print('귀무가설 기각, 4개조의 불량수는 차이가 있다.')
    
p_val = 1-chi_.cdf(chi_squared_statistic)
print('p-value:', p_val)

#유의수준 기반 검정
if p_val > 0.05:
    print('귀무가설 채택, 4개조의 불량수는 차이가 없다.')
else:
    print('귀무가설 기각, 4개조의 불량수는 차이가 있다.')

귀무가설 기각, 4개조의 불량수는 차이가 있다.
p-value: 0.04438695936738335
귀무가설 기각, 4개조의 불량수는 차이가 있다.


In [8]:
# chisquare " 모집단의 분포를 알고있고, 이에 대한 검증을 할 때, 활용"

stats.chisquare(ct_1.loc['관측빈도'].values, f_exp = ct_1.loc['기대빈도'].values)

Power_divergenceResult(statistic=8.079999999999998, pvalue=0.044386959367383315)

In [9]:
#교차분석하기 위한 범중형 데이터 정의
cross_data= pd.DataFrame({"영양제": ['복용','복용','복용','복용','복용','복용','복용','복용','복용','복용','복용','복용','복용','복용','복용','복용','복용','복용','복용','복용',
                                  '미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용','미복용',],
                          "감기여부" : ['유','유','유','유','유','무','무','무','무','무',
                 '무','무','무','무','무','무','무','무','무','무',
                 '유','유','유','유','유','유','유','유','유','유',
                 '유','유','무','무','무','무','무','무','무','무', ]
                          }
                         
                         )
cross_data.head(5)

,영양제,감기여부
0,복용,유
1,복용,유
2,복용,유
3,복용,유
4,복용,유


In [10]:
# 교차분석표 생성
print(pd.crosstab(cross_data['영양제'], cross_data['감기여부'], margins=True))

# 계산하기 위한 교차표
data_crosstab = pd.crosstab(cross_data['영양제'], cross_data['감기여부'],margins=False)


감기여부   무   유  All
영양제              
미복용    8  12   20
복용    15   5   20
All   23  17   40


In [11]:
data_crosstab

감기여부,무,유
영양제,,
미복용,8,12
복용,15,5


In [12]:
# 교차표분석 : 카이제곱 통계량 산출
from scipy.stats import chi2_contingency

result = chi2_contingency(observed=data_crosstab, correction=False)
print('1. 카이제곱 통계량:', result[0])
print('2. p-value:', result[1])
print('3. df::', result[2])     # 행의개수-1 * 열의개수-1
print("4. 기대값 행렬:\n", pd.DataFrame(result[3]).rename(index={0:'복용',1:'미복용'},columns={0:'유', 1:'무'}))
if 0.05 >= result[1]: # p-value
    print('귀무가설 기각, 감기약 복용과 감기 유무는 차이가 있다.')
else : 
    print('귀무가설 채택, 감기약 복용과 감기 유무는 차이가 없다')

1. 카이제곱 통계량: 5.012787723785166
2. p-value: 0.025160759200408785
3. df:: 1
4. 기대값 행렬:
         유    무
복용   11.5  8.5
미복용  11.5  8.5
귀무가설 기각, 감기약 복용과 감기 유무는 차이가 있다.
